# Homework 4 (Ibotta Products)

In [1]:
# import analysis packages
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [93]:
# read data from .csv files
trainDF = pd.read_csv('./ibotta_train.csv')
testDF = pd.read_csv('./ibotta_test.csv')

# drop category code field
trainDF.drop('Cat_code', axis = 1, inplace = True)

# combine data sets for preprocessing
trainDF['origin'] = 'train'
testDF['origin'] = 'test'
fullDF = pd.concat([trainDF, testDF])

# text cleaning
fullDF['Brand_name'].where(-fullDF['Brand_name'].isna(), '', inplace = True)
fullDF['Brand_name'] = fullDF['Brand_name'].apply(lambda x: x.lower().replace("'", "").replace(",", "").replace(":", "").replace("-", "").replace(".", ""))
fullDF['Name'] = fullDF['Name'].apply(lambda x: x.lower().replace("'", "").replace(",", "").replace(":", "").replace("-", "").replace(".", ""))

# combine brand and name fields
fullDF['brandAlready'] = fullDF.apply(lambda x: x['Name'].find(x['Brand_name']), axis = 1)
fullDF.loc[fullDF.brandAlready == -1, 'Name'] = fullDF.loc[fullDF.brandAlready == -1, 'Brand_name'] + \
    ' ' + fullDF.loc[fullDF.brandAlready == -1, 'Name']
fullDF.drop('brandAlready', axis = 1, inplace = True)

# split data
trainDF = pd.DataFrame(fullDF.loc[fullDF['origin'] == 'train'].drop('origin', axis = 1))
testDF = pd.DataFrame(fullDF.loc[fullDF['origin'] == 'test'].drop(['origin', 'Category'], axis = 1))

# check dimensions of data
display(trainDF.shape)
display(testDF.shape)


# check data types
display(trainDF.info())

# peek at data
display(trainDF.head())

(8000, 4)

(1999, 3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Id          8000 non-null   int64 
 1   Name        8000 non-null   object
 2   Brand_name  8000 non-null   object
 3   Category    8000 non-null   object
dtypes: int64(1), object(3)
memory usage: 312.5+ KB


None

,Id,Name,Brand_name,Category
0,1,4c homestyle parmesan grated cheese 6 oz,4c,Dairy
1,2,4c 100% natural parmesan cheese 6 oz,,Dairy
2,3,4c parmesan & romano cheese grated,4c,Dairy
3,4,advance fast fixin country fried steaks with g...,fast fixin,Frozen Foods
4,5,borden dairy company borden salted 4 ct butter,borden dairy company,Dairy


In [85]:
pd.DataFrame(fullDF.loc[fullDF['origin'] == 'train'].drop('origin', axis = 1, inplace = True))

/var/folders/2q/4pj4sjs90_vd2_z1qkz6xp3w0000gn/T/ipykernel_79290/234618447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd.DataFrame(fullDF.loc[fullDF['origin'] == 'train'].drop('origin', axis = 1, inplace = True))


""
